Our project topic:
Back-test a rotation strategy based on aggregate valuations of the NIFTY non-financial sector indices (AUTO, COMMODITIES, FMCG, etc...)

How we have planned to go about it:

1. Taking 5-6 non-financial sector indices like Auto, commodities, metal, fmcg,pharma.

2. Determine seasonality and cyclicity in the each of these chosen indices. 

3. Based on this information, determine the frequency and the factors to be considered for rotation. 

4. Performing 5 different rotation strategies like:
Fixed ratio
Beta based
Variable ratio
Constant rupee plan
Constant ratio plan
This will help us understand the timing of entry and exit and the composition of the portfolio yielding maximum returns. 

5. Comparing the returns form each of these five types of strategies with that of a portfolio without any rotation IE, an untouched portfolio. 

In [1]:
library(dbplyr)
library(dplyr)
library(odbc)
library(plutoR)
options("scipen"=999)
source("config.R")

indices<-Indices()


Attaching package: ‘dplyr’

The following objects are masked from ‘package:dbplyr’:

    ident, sql

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [2]:
indices$IndiaVixTimeSeries() %>%
  top_n(10, wt=TIME_STAMP) %>%
  print(n=Inf)

ERROR: Error: <SQL> 'SELECT *
FROM (SELECT TOP 100 PERCENT "TIME_STAMP", "PX_HIGH" AS "HIGH", "PX_LOW" AS "LOW", "PX_OPEN" AS "OPEN", "PX_CLOSE" AS "CLOSE"
FROM "VIX_HISTORY") "dbplyr_001"
WHERE (top_n_rank(10.0, "TIME_STAMP"))'
  nanodbc/nanodbc.cpp:1587: 42000: [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]'top_n_rank' is not a recognized built-in function name. 


In [3]:
indices$IndiaVixTimeSeries() %>%
  top_n(10, wt=TIME_STAMP) %>%
  print(n=Inf)

ERROR: Error: <SQL> 'SELECT *
FROM (SELECT TOP 100 PERCENT "TIME_STAMP", "PX_HIGH" AS "HIGH", "PX_LOW" AS "LOW", "PX_OPEN" AS "OPEN", "PX_CLOSE" AS "CLOSE"
FROM "VIX_HISTORY") "dbplyr_002"
WHERE (top_n_rank(10.0, "TIME_STAMP"))'
  nanodbc/nanodbc.cpp:1587: 42000: [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]'top_n_rank' is not a recognized built-in function name. 


This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)